## Farmers Problems
Birge, J. R., Louveaux, F., Birge, J. R., & Louveaux, F. (2011). Introduction and examples. Introduction to Stochastic Programming, 3-54.

In [21]:
import pyomo.environ as pyo

In [22]:
def build_farmers():
    m = pyo.ConcreteModel()

    m.x1 = pyo.Var(within=pyo.NonNegativeReals) # acres of land devoted to wheat
    m.x2 = pyo.Var(within=pyo.NonNegativeReals) # acres of land devoted to corn
    m.x3 = pyo.Var(within=pyo.NonNegativeReals) # acres of land devoted to sugar beets

    m.y1 = pyo.Var(within=pyo.NonNegativeReals) # tons of wheat to purchased
    m.y2 = pyo.Var(within=pyo.NonNegativeReals) # tons of corn to purchased

    m.w1 = pyo.Var(within=pyo.NonNegativeReals) # tons of wheat to sold
    m.w2 = pyo.Var(within=pyo.NonNegativeReals) # tons of corn to sold
    m.w3 = pyo.Var(within=pyo.NonNegativeReals) # tons of sugar beets to sold at a favorable price
    m.w4 = pyo.Var(within=pyo.NonNegativeReals) # tons of sugar beets to sold at a lower price

    def obj_rule(m):
        return 150*m.x1 + 230*m.x2 + 260*m.x3 + 238*m.y1 - 170*m.w1 + 210*m.y2 - 150*m.w2 - 36*m.w3 - 10*m.w4
    
    m.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    def cons1_rule(m):
        return m.x1 + m.x2 + m.x3 <= 500
    
    m.cons1 = pyo.Constraint(rule=cons1_rule)

    def cons2_rule(m):
        return 2.5 * m.x1 + m.y1 - m.w1 >= 200
    
    m.cons2 = pyo.Constraint(rule=cons2_rule)

    def cons3_rule(m):
        return 3 * m.x2 + m.y2 - m.w2 >= 240
    
    m.cons3 = pyo.Constraint(rule=cons3_rule)

    def cons4_rule(m):
        return m.w3 + m.w4 <= 20 * m.x3
    
    m.cons4 = pyo.Constraint(rule=cons4_rule)

    def cons5_rule(m):
        return m.w3 <= 6000
    
    m.cons5 = pyo.Constraint(rule=cons5_rule)

    return m

m = build_farmers()
solver = pyo.SolverFactory('gurobi')
solver.solve(m)

print("Acres of land devoted to wheat: ", pyo.value(m.x1))
print("Acres of land devoted to corn: ", pyo.value(m.x2))
print("Acres of land devoted to sugar beets: ", pyo.value(m.x3))
print("Tons of wheat to purchased: ", pyo.value(m.y1))
print("Tons of corn to purchased: ", pyo.value(m.y2))
print("Tons of wheat to sold: ", pyo.value(m.w1))
print("Tons of corn to sold: ", pyo.value(m.w2))
print("Tons of sugar beets to sold at a favorable price: ", pyo.value(m.w3))
print("Tons of sugar beets to sold at a lower price: ", pyo.value(m.w4))
print("Profit: ", -pyo.value(m.obj))

Acres of land devoted to wheat:  120.0
Acres of land devoted to corn:  80.0
Acres of land devoted to sugar beets:  300.0
Tons of wheat to purchased:  0.0
Tons of corn to purchased:  0.0
Tons of wheat to sold:  100.0
Tons of corn to sold:  0.0
Tons of sugar beets to sold at a favorable price:  6000.0
Tons of sugar beets to sold at a lower price:  0.0
Profit:  118600.0


In [30]:
# Problem 4
def build_stochastic_farmers():
    m = pyo.ConcreteModel()
    I = [1, 2, 3]  # Scenarios

    # Decision variables
    m.x1 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of land for wheat
    m.x2 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of land for corn
    m.x3 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of land for sugar beets

    m.y1 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of wheat purchased
    m.y2 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of corn purchased

    m.w1 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of wheat sold
    m.w2 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of corn sold
    m.w3 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of sugar beets sold at a favorable price
    m.w4 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of sugar beets sold at a lower price

    # Objective function
    def obj_rule(m):
        # Ensure that coefficients correctly reflect costs and revenues for each scenario
        return sum([150*m.x1, 230*m.x2, 260*m.x3]) + sum([1/3 * (238*m.y1[i] - 170*m.w1[i] + 210*m.y2[i] - 150*m.w2[i] - 36*m.w3[i] - 10*m.w4[i]) for i in I])
    
    m.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # Constraints
    m.cons1 = pyo.Constraint(expr=m.x1 + m.x2 + m.x3 <= 500)  # Total land constraint

    # Wheat production/purchase constraints for each scenario
    m.cons21 = pyo.Constraint(expr=3 * m.x1 + m.y1[1] - m.w1[1] >= 200)
    m.cons2 = pyo.Constraint(expr=2.5 * m.x1 + m.y1[2] - m.w1[2] >= 200)
    m.cons22 = pyo.Constraint(expr=2 * m.x1 + m.y1[3] - m.w1[3] >= 200)

    # Corn production/purchase constraints for each scenario
    m.cons31 = pyo.Constraint(expr=3.6 * m.x2 + m.y2[1] - m.w2[1] >= 240)
    m.cons3 = pyo.Constraint(expr=3 * m.x2 + m.y2[2] - m.w2[2] >= 240)
    m.cons32 = pyo.Constraint(expr=2.4 * m.x2 + m.y2[3] - m.w2[3] >= 240)

    # Sugar beets sale constraints for each scenario
    m.cons41 = pyo.Constraint(expr=m.w3[1] + m.w4[1] <= 24 * m.x3)
    m.cons4 = pyo.Constraint(expr=m.w3[2] + m.w4[2] <= 20 * m.x3)
    m.cons42 = pyo.Constraint(expr=m.w3[3] + m.w4[3] <= 16 * m.x3)

    m.cons5 = pyo.Constraint(I, rule=lambda m, i: m.w3[i] <= 6000)  # Sugar beets favorable price sale limit

    return m

m = build_stochastic_farmers()
solver = pyo.SolverFactory('gurobi')
solver.solve(m)

# Print results
print("Acres of land devoted to wheat: ", pyo.value(m.x1))
print("Acres of land devoted to corn: ", pyo.value(m.x2))
print("Acres of land devoted to sugar beets: ", pyo.value(m.x3))
print("Tons of wheat to purchased: ", [pyo.value(m.y1[i]) for i in m.y1])
print("Tons of corn to purchased: ", [pyo.value(m.y2[i]) for i in m.y2])
print("Tons of wheat to sold: ", [pyo.value(m.w1[i]) for i in m.w1])
print("Tons of corn to sold: ", [pyo.value(m.w2[i]) for i in m.w2])
print("Tons of sugar beets to sold at a favorable price: ", [pyo.value(m.w3[i]) for i in m.w3])
print("Tons of sugar beets to sold at a lower price: ", [pyo.value(m.w4[i]) for i in m.w4])
print("Profit: ", -pyo.value(m.obj))

Acres of land devoted to wheat:  170.0
Acres of land devoted to corn:  80.0
Acres of land devoted to sugar beets:  250.0
Tons of wheat to purchased:  [0.0, 0.0, 0.0]
Tons of corn to purchased:  [0.0, 0.0, 48.00000000000003]
Tons of wheat to sold:  [310.0, 225.0, 140.0]
Tons of corn to sold:  [48.0, 0.0, 0.0]
Tons of sugar beets to sold at a favorable price:  [6000.0, 5000.0, 4000.0]
Tons of sugar beets to sold at a lower price:  [0.0, 0.0, 0.0]
Profit:  108390.0
